## Downloading all the required libraries

In [1]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json #

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0

The following packages will be UPDATED:

  openssl                                 1.1.1f-h516909a_0 --> 1.1.1g-h516909a_0



openssl-1.1.1g       | 2.1 MB    | ############

## Loading the dataset containing the neighborhoods of mumbai

In [2]:
df_mumbai = pd.read_csv('MumbaI_Neighborhood_data (1).csv')

In [3]:
df_mumbai.head()

,Unnamed: 0,Location,Pincode,State,District,Latitude,Longitude
0,NaN,A I staff colony,400029,Maharashtra,Mumbai,19.078700,72.860050
1,NaN,Aareymilk Colony,400065,Maharashtra,Mumbai,19.148509,72.881740
2,NaN,Agripada,400011,Maharashtra,Mumbai,18.984810,72.816860
3,NaN,Airport,400099,Maharashtra,Mumbai,19.090052,72.868672
4,NaN,Ambewadi,400004,Maharashtra,Mumbai,18.963010,72.823040


In [4]:
df_mumbai.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
df_mumbai.head()

,Location,Pincode,State,District,Latitude,Longitude
0,A I staff colony,400029,Maharashtra,Mumbai,19.078700,72.860050
1,Aareymilk Colony,400065,Maharashtra,Mumbai,19.148509,72.881740
2,Agripada,400011,Maharashtra,Mumbai,18.984810,72.816860
3,Airport,400099,Maharashtra,Mumbai,19.090052,72.868672
4,Ambewadi,400004,Maharashtra,Mumbai,18.963010,72.823040


## Finding the latitude and longitude of Mumbai

In [6]:
address = 'Mumbai'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai are 18.9387711, 72.8353355.


## Creating a map

In [7]:
map_mum = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, loca, pin in zip(df_mumbai['Latitude'], df_mumbai['Longitude'], df_mumbai['Location'], df_mumbai['Pincode']):
    label = '{}, {}'.format(loca, pin)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mum)  
    
map_mum

## Foursquare Credentials and Version

In [8]:
CLIENT_ID = 'J1QUBNPCUUS3TGMKMY12UEMMVN0ZZAB2SSBD4OJ3P0V4GPKW' # your Foursquare ID
CLIENT_SECRET = 'UW3NKX4SWAXJNFWVOASG2ECYGXCDES2B2BWL5MZXDLULMFRS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: J1QUBNPCUUS3TGMKMY12UEMMVN0ZZAB2SSBD4OJ3P0V4GPKW
CLIENT_SECRET:UW3NKX4SWAXJNFWVOASG2ECYGXCDES2B2BWL5MZXDLULMFRS


## Getting Nearby Venues

In [9]:
radius=500
LIMIT=100
venues_list=[]
    
for lat,lng,loca,post in zip(df_mumbai['Latitude'], df_mumbai['Longitude'], df_mumbai['Location'], df_mumbai['Pincode']):
            
        
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        
    results = requests.get(url).json()["response"]['groups'][0]['items']
        
    for v in results:
        venues_list.append((
            post, 
            loca,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']))   


In [10]:
df_with_venues = pd.DataFrame(venues_list)
df_with_venues.columns = ['Pincode', 'Location', 'Latitude', 'Longitude', 'Venue_Name', 'Venue_Latitude', 'Venue_Longitude', 'Venue_Category']
df_with_venues.head()

,Pincode,Location,Latitude,Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category
0,400029,A I staff colony,19.07870,72.86005,Natural's Ice Cream,19.077560,72.863035,Ice Cream Shop
1,400029,A I staff colony,19.07870,72.86005,Geeta Vihar Hotel,19.078022,72.862714,Indian Restaurant
2,400029,A I staff colony,19.07870,72.86005,Kalina bus stop,19.076665,72.862189,Bus Station
3,400029,A I staff colony,19.07870,72.86005,Chetan Punjab,19.081774,72.857418,Bar
4,400011,Agripada,18.98481,72.81686,Gallops,18.981231,72.817178,Restaurant


## Number of venues per Location

In [11]:
df_with_venues.groupby("Location").count()

,Pincode,Latitude,Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category
Location,,,,,,,
A I staff colony,4,4,4,4,4,4,4
Agripada,12,12,12,12,12,12,12
Airport,4,4,4,4,4,4,4
Ambewadi,4,4,4,4,4,4,4
Andheri,19,19,19,19,19,19,19
Andheri East,12,12,12,12,12,12,12
Andheri Railway station,12,12,12,12,12,12,12
Antop Hill,4,4,4,4,4,4,4
Asvini,1,1,1,1,1,1,1


## Check if there are Italian Restaurants in the Venue Categories

In [12]:
"Italian Restaurant" in df_with_venues['Venue_Category'].unique()

True

## Analysing Locations

In [13]:
mum_onehot = pd.get_dummies(df_with_venues[['Venue_Category']], prefix="", prefix_sep="")
mum_onehot['Location'] = df_with_venues['Location']
fixed_columns = [mum_onehot.columns[-1]] + list(mum_onehot.columns[:-1])
mum_onehot = mum_onehot[fixed_columns]
mum_onehot.head()

,Location,ATM,Accessories Store,Airport,Airport Lounge,Airport Terminal,Amphitheater,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Bistro,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Bus Station,Business Service,Café,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Donut Shop,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gujarati Restaurant,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Indoor Play Area,Intersection,Irani Cafe,Italian Restaurant,Japanese Restaurant,Juice Bar,Lake,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,National Park,New American Restaurant,Nightclub,Noodle House,Opera House,Other Great Outdoors,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Spa,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Tea Room,Tennis Court,Thai Restaurant,Theater,Trail,Train Station,Travel Lounge,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,A I staff colony,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,A I staff colony,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,A I staff colony,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,A I staff colony,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Agripada,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

## Group rows by location and by taking the mean of the frequency of occurrence of each category

In [14]:
mum_grouped = mum_onehot.groupby(['Location']).mean().reset_index()
mum_grouped.head()

,Location,ATM,Accessories Store,Airport,Airport Lounge,Airport Terminal,Amphitheater,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Bistro,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Bus Station,Business Service,Café,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Donut Shop,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gujarati Restaurant,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Indoor Play Area,Intersection,Irani Cafe,Italian Restaurant,Japanese Restaurant,Juice Bar,Lake,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,National Park,New American Restaurant,Nightclub,Noodle House,Opera House,Other Great Outdoors,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Spa,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Tea Room,Tennis Court,Thai Restaurant,Theater,Trail,Train Station,Travel Lounge,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,A I staff colony,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.25,0.0,0.250000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,Agripada,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.000000,0.00,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.083333,0.000000,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.

## Only Italian Restaurants

In [15]:
mum_italian = mum_grouped[['Location', 'Italian Restaurant']]
mum_italian.head()

,Location,Italian Restaurant
0,A I staff colony,0.000000
1,Agripada,0.083333
2,Airport,0.000000
3,Ambewadi,0.000000
4,Andheri,0.000000


## Clustering

In [16]:
kclusters = 4
mum_cluster = mum_italian.drop('Location', axis=1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mum_cluster)
kmeans.labels_[0:10]


array([0, 3, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

## Creating new dataframe

In [19]:
mum_merged = mum_italian.copy()
mum_merged['Cluster Labels'] = kmeans.labels_
mum_merged = mum_merged.join(df_with_venues.set_index('Location'), on='Location')



In [20]:
mum_merged.head()

,Location,Italian Restaurant,Cluster Labels,Pincode,Latitude,Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category
0,A I staff colony,0.000000,0,400029,19.07870,72.86005,Natural's Ice Cream,19.077560,72.863035,Ice Cream Shop
0,A I staff colony,0.000000,0,400029,19.07870,72.86005,Geeta Vihar Hotel,19.078022,72.862714,Indian Restaurant
0,A I staff colony,0.000000,0,400029,19.07870,72.86005,Kalina bus stop,19.076665,72.862189,Bus Station
0,A I staff colony,0.000000,0,400029,19.07870,72.86005,Chetan Punjab,19.081774,72.857418,Bar
1,Agripada,0.083333,3,400011,18.98481,72.81686,Gallops,18.981231,72.817178,Restaurant


## Visualizing Clusters

In [21]:

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)


x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, loca, cluster in zip(mum_merged['Latitude'],mum_merged['Longitude'], mum_merged['Location'], mum_merged['Cluster Labels']):
    label = folium.Popup(str(loca) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examining Clusters

In [22]:
mum_merged.loc[(mum_merged['Cluster Labels'] == 0) & (mum_merged['Venue_Category'] == 'Italian Restaurant')]

,Location,Italian Restaurant,Cluster Labels,Pincode,Latitude,Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category
98,Mantralaya,0.013158,0,400032,18.929015,72.830121,Relish,18.93032,72.826361,Italian Restaurant


In [23]:
mum_merged.loc[(mum_merged['Cluster Labels'] == 1) & (mum_merged['Venue_Category'] == 'Italian Restaurant')]

,Location,Italian Restaurant,Cluster Labels,Pincode,Latitude,Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category
41,Danda,0.130435,1,400052,19.074707,72.825325,Out of the Blue,19.071756,72.825535,Italian Restaurant
41,Danda,0.130435,1,400052,19.074707,72.825325,Shatranj Napoli,19.071727,72.824904,Italian Restaurant
41,Danda,0.130435,1,400052,19.074707,72.825325,Pastaa Bastta,19.070723,72.823117,Italian Restaurant
113,Nariman Point,0.116279,1,400021,18.928390,72.822350,The Bayview - Hotel Marine Plaza,18.931876,72.823031,Italian Restaurant
113,Nariman Point,0.116279,1,400021,18.928390,72.822350,Relish,18.930320,72.826361,Italian Restaurant
113,Nariman Point,0.116279,1,400021,18.928390,72.822350,Ristorante Prego,18.926274,72.822642,Italian Restaurant
113,Nariman Point,0.116279,1,400021,18.928390,72.822350,Spesso Gourmet Kitchen,18.924439,72.821560,Italian Restaurant
113,Nariman Point,0.116279,1,400021,18.928390,72.822350,Vetro,18.926731,72.820572,Italian Restaurant
115,New Yogakshema,0.116279,1,400021,18.928390,72.822350,The Bayview - Hotel Marine Plaza,18.931876,72.823031,Italian Restaurant
115,New Yogakshema,0.116279,1,400021,18.928390,72.822350,Relish,18.930320,72.826361,Italian Restaurant


In [24]:
mum_merged.loc[(mum_merged['Cluster Labels'] == 2) & (mum_merged['Venue_Category'] == 'Italian Restaurant')]

,Location,Italian Restaurant,Cluster Labels,Pincode,Latitude,Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category
12,B.P.lane,0.023810,2,400003,18.958222,72.813189,New Yorker,18.955360,72.813698,Italian Restaurant
17,Bharat Nagar,0.041667,2,400007,18.956350,72.815420,New Yorker,18.955360,72.813698,Italian Restaurant
17,Bharat Nagar,0.041667,2,400007,18.956350,72.815420,Little Italy,18.954814,72.817279,Italian Restaurant
26,Chamarbaug,0.032258,2,400012,19.000800,72.830850,Pronto,19.002965,72.828912,Italian Restaurant
48,Girgaon,0.037037,2,400004,18.955966,72.819045,Little Italy,18.954814,72.817279,Italian Restaurant
53,Grant Road,0.041667,2,400007,18.956350,72.815420,New Yorker,18.955360,72.813698,Italian Restaurant
53,Grant Road,0.041667,2,400007,18.956350,72.815420,Little Italy,18.954814,72.817279,Italian Restaurant
55,Haffkin Institute,0.032258,2,400012,19.000800,72.830850,Pronto,19.002965,72.828912,Italian Restaurant
59,High Court bulding,0.043478,2,400032,18.929380,72.827500,Salt Water Cafe,18.933500,72.826300,Italian Restaurant
59,High Court bulding,0.043478,2,400032,18.929380,72.827500,Relish,18.930320,72.826361,Italian Restaurant


In [25]:
mum_merged.loc[(mum_merged['Cluster Labels'] == 3) & (mum_merged['Venue_Category'] == 'Italian Restaurant')]

,Location,Italian Restaurant,Cluster Labels,Pincode,Latitude,Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category
1,Agripada,0.083333,3,400011,18.984810,72.816860,Little Italy,18.983885,72.815241,Italian Restaurant
32,Churchgate,0.060000,3,400020,18.932255,72.826428,Salt Water Cafe,18.933500,72.826300,Italian Restaurant
32,Churchgate,0.060000,3,400020,18.932255,72.826428,The Bayview - Hotel Marine Plaza,18.931876,72.823031,Italian Restaurant
32,Churchgate,0.060000,3,400020,18.932255,72.826428,Relish,18.930320,72.826361,Italian Restaurant
56,Haines Road,0.083333,3,400011,18.984810,72.816860,Little Italy,18.983885,72.815241,Italian Restaurant
57,Hajiali,0.052632,3,400034,18.977454,72.810920,Pronto,18.975295,72.812783,Italian Restaurant
66,Jacob Circle,0.083333,3,400011,18.984810,72.816860,Little Italy,18.983885,72.815241,Italian Restaurant
78,Khar Delivery,0.062500,3,400052,19.075550,72.832060,Spaghetti Kitchen,19.072907,72.831963,Italian Restaurant
138,Santacruz(east),0.055556,3,400055,19.078700,72.850000,Celini,19.076831,72.851734,Italian Restaurant
151,Tulsiwadi,0.062500,3,400034,18.974541,72.814242,Pronto,18.975295,72.812783,Italian Restaurant


# As we can see above, clusters 0 is the best place to open an Italian Restaurant as it has the least amount of Italian restaurants